In [1]:
import pickle
def read_bin(filename):
    with open(filename,'rb') as bf:
        bin_data = pickle.load(bf)
    return bin_data

def save_bin(filename,data):
    with open(filename,'wb') as bf:
        pickle.dump(data,bf)

In [2]:
# gpt35_df=read_bin("new_data/gpt35_df.bin")
# gpt4o_df=read_bin("new_data/gpt4o_df.bin")
gpt4_df=read_bin("new_data/gpt4_df.bin")

In [3]:
# del gpt35_df
# del gpt4o_df
# del gpt4_df

In [4]:
FABLE = read_bin("new_data/FABLE.bin")

In [5]:
gpt4_df.head(1)

,id,datePublished,itemReviewed.datePublished,claimReviewed,author.name,reviewRating.@type,reviewRating.ratingValue,reviewRating.alternateName,date_diff,language
25,75eae0a2-c474-4200-9889-f1f80a83be5e,2024-04-09 00:00:00+00:00,2024-03-27 00:00:00+00:00,Laut ORF-Artikel wurde Siegfried Meryn in eine...,dpa,Rating,NaN,"Die angebliche ORF-Webseite wurde gefälscht, d...",13.0,de


In [6]:
def duplicate_checker(df):
    if df['id'].duplicated().any():
        print("There are duplicate IDs. Here they are:")
        # 重複している行を表示
        print(df[df['id'].duplicated(keep=False)])
    else:
        print("No duplicate IDs found.")
        
duplicate_checker(gpt4_df)

No duplicate IDs found.


## GPT使用

In [7]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['openai']['api_key']

In [8]:
import openai
openai.api_key = api_key

In [9]:
# def gpt_ans(claim,question):
#     prompt = "Read the following claim and evaluate the question provided. Return the answer as <answer>Yes, No, or Unknown</answer>. claim: "+claim+" Question: "+ question+". Please explain your reasoning in <reason>tags</reason>."
#     response = openai.ChatCompletion.create(
# #       model="gpt-3.5-turbo-0125",  # 2023年の最新のGPT-3.5モデル名に基づく
#       model="gpt-4-turbo",
#       messages=[
#           {"role": "user", "content":prompt}
#       ]
#     )

#     return response['choices'][0]['message']['content']  # 生成されたテキストを表示

In [10]:
import time
from requests.exceptions import RequestException

def gpt_ans(claim, question):
    prompt = "Read the following claim and evaluate the question provided. Return the answer as <answer>Yes, No, or Unknown</answer>. claim: "+claim+" Question: "+ question+". Please explain your reasoning in <reason>tags</reason>."
    for i in range(3):# 5回までリトライ
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            return response['choices'][0]['message']['content']
        except openai.error.APIError as e:
            time.sleep(5)  # 5秒待ってから再試行
            if i == 2:
                return ""
        except RequestException as e:
            time.sleep(5)
            if i == 2:
                return ""

In [11]:
gpt4_df_id = list(gpt4_df["id"])
gpt4_df_claimReviewed = list(gpt4_df["claimReviewed"])

In [13]:
ans_dict = dict()

In [25]:
c = 2174
for c_id, claim in zip(gpt4_df_id[c:3000],gpt4_df_claimReviewed[c:3000]):

    ans = dict()
    for q,question in enumerate(FABLE):
        print('\r%d / %d  ( %d / %d )' %(c, 6718, q, 52), end='')
        ans[q]=gpt_ans(claim,question)
        
    ans_dict[c_id] = ans
    
    if c % 10 == 0:
        save_bin("new_data/ans_dict_2001-2999/ans_dict_-"+str(c)+".bin",ans_dict)
        ans_dict = dict()
    c += 1

2200 / 6718  ( 13 / 52 )

KeyboardInterrupt: 

In [202]:
for k,v in ans_dict.items():
    print(k)
    print(len(v))

90b94083-3c3e-474a-8d97-7798c0b89589
52
df3ac55f-9050-4022-97c0-d8e2d83b00db
52
48a3e384-7594-476c-bb85-5a458e0ed3e2
52
39faad84-5fcd-471d-9f3b-c0ba4fe9e14f
52
d7208c89-4a08-4877-ac34-ab913bea2155
52


In [203]:
gpt4_df_id[3000:3006]

['90b94083-3c3e-474a-8d97-7798c0b89589',
 'df3ac55f-9050-4022-97c0-d8e2d83b00db',
 '48a3e384-7594-476c-bb85-5a458e0ed3e2',
 '39faad84-5fcd-471d-9f3b-c0ba4fe9e14f',
 'd7208c89-4a08-4877-ac34-ab913bea2155',
 '0420b7e1-d582-41ad-a962-854aea60fcfd']

In [199]:
c

3005

In [200]:
save_bin("new_data/ans_dict_3000-/ans_dict_-"+str(c)+".bin",ans_dict)

In [196]:
ans_dict["df3ac55f-9050-4022-97c0-d8e2d83b00db"]

{0: "<answer>Unknown</answer>\n<reason>The information provided in the claim does not give enough context to determine if it fits into a larger story or argument about how the world works or people think. The claim only indicates the content of a video where Hillary Clinton purportedly blames her election loss and Jeffrey Epstein's death on climate world and people's beliefs. Without additional context or understanding the reasons behind her statements, it is difficult to evaluate how it connects to larger narratives or societal views.</reason>",
 1: '<answer>No</answer>\n<reason>The claim focuses on attributing certain unrelated events to climate change as stated by Hillary Clinton according to the video, but it does not inherently question the trust in or the functioning of public institutions. The specifics of her blaming these events on climate change might indicate irrational or misleading arguments or could discuss her perception of climate impact widely and inappropriately, but 

In [177]:
len(ans_dict["0ed308be-7eb4-4318-8a86-b93a9bbc047f"])

KeyError: '0ed308be-7eb4-4318-8a86-b93a9bbc047f'

In [170]:
for q,question in enumerate(FABLE[:3]):
    print(q)

0
1
2


In [122]:
# ans_dict[14]

In [89]:
claim_text

['Volodymyr Zelensky a acheté la Highgrove House du roi Charles III pour 20 millions de livres',
 '*வயநாட்டில் காங்கிரஸ் ராகுல் காந்தியை ஆதரித்து பாகிஸ்தான் கொடியை அசைத்து ஆதரவு தெரிவிக்கிறார்கள். தயவு செய்து திருந்துங்கள்.*',
 'DEUTSCHER GEWINNT BEIM EU GERICHT: FINGERABDRUCK AUF PERSONALAUSWEIS AB 2027 UNGÜLTIG',
 'Le chef des renseignements ukrainiens, Kyrylo Boudanov, s\'est dit "très heureux" suite à l\'attaque terroriste du Crocus City Hall à Moscou.',
 'Este video muestra un bombardeo iraní en Tel Aviv (Israel). ',
 'Este video muestra el eclipse solar presenciado en Norteamérica el 8 de abril de 2024.',
 'Este video muestra cómo censistas borran los datos en el Censo 2024 realizado en Chile. ',
 'Eine fehlende Ecke am Stimmzettel und ein Bleistift zum Ankreuzen machen einen Stimmzetteln ungültig.',
 'திமுக மாவட்டச் செயலாளர் சிற்றரசு வீட்டில் அமலாக்கத் துறை அதிகாரிகள் போதை மருந்து கடத்தல் தொடர்பாக சோதனை.']

In [50]:
counter = 0
for index, row in gpt35_df.iterrows():
    if counter < 10:  # 3回のみ繰り返す
        print(row['claimReviewed'])
        print()
        ans = []
        for question in FABLE[0]:
            print(question)
        counter += 1

Benjamin Netanyahu não atendeu Lula ao telefone, indignado com a simpatia do brasileiro ao Hamas, segundo o Embaixador de Israel no Brasil, Daniel Zonshine. O embaixador ainda disse que o Brasil governado por Lula está incluso na lista negra de Países que apoiam o Terrorismo.

Does the message fit into a larger story or argument, for example about how the world works or how people think?
Does the message question trust in or the functioning of public institutions?
Das Video zeigt ein brennendes Polizeiauto in Schweden.

Does the message fit into a larger story or argument, for example about how the world works or how people think?
Does the message question trust in or the functioning of public institutions?
